# Introduction to Dynamic Programming

**Why learn dynamic programming?**

1. **Policy recommendations** are central for all social sciences
2. Need models for **counter-factual analysis**
3. Realistic models $\rightarrow$ **no analytical solution**
  1. dynamic
  2. multi-dimensional
  3. uncertainty
  4. heterogeneity
  5. learning
4. **Solving such models requires dynamic programming**
5. Estimation gives you **laboratory economies**

**This mini-course:**

1. Go through this notebook, `01. Introduction.ipynb`
2. Go through the notebook, `02. Your First Consumption-Saving Model`
3. Read through the slides `ConsumptionSaving.pdf`
4. Go through `03. A Full Blow Consumption Model.ipynb`
5. Go through `04. Structural Estimation.ipynb`
6. Go through `05. General equilibrium.ipynb`

**Requirements:** You are assumed to know basic Python programming. Else see [Introduction to Programming and Numerical Analysis](https://numeconcopenhagen.netlify.com/).

**Intended learning goal:** You are will be equipped to solve state-of-the-art consumption saving models, estimate them and embed them in general equilibrium models.

**Books:**

1. Stockey and Lucas (1989), Recursive Methods in Economics
2. Judd (1998): Numerical Methods
3. Bertsekas (2005, 2012): Dynamic Programming and Optimal Control Vol. I-II
4. Puterman (2008), Markov Decision Processes: Discrete-Stochastic Dynamic Programming
5. Powell (2011), Approximate Dynamic Programming: Solving the Curses of Dimensionality

**Online material:**

1. [Quantitative Economics](https://lectures.quantecon.org/)
2. [Scipy lecture notes](https://scipy-lectures.org/)
3. [Econ-Ark](https://econ-ark.org/)
4. [Lecture notes by Fernandez-Villaverde](https://www.sas.upenn.edu/~jesusfv/teaching.html)
5. [Lecture notes by Fatih Guvenen](https://fatihguvenen.com/teaching/econ8185-phd-computation-empirics/)
6. [Lecture notes by Gianluca Violante](https://sites.google.com/a/nyu.edu/glviolante/teaching/quantmacro15)
7. [Lecture notes by Wouter J. den Haan](http://www.wouterdenhaan.com/notes.htm)

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize

from consav.misc import elapsed

# Bellmans Principle of Optimality

Consider a consumer who:

1. Live in $T$ periods.
2. Have $M$ apples.
3. Can only consumer a natural number of apples in each period, $C_t \in \mathbb{N}, \, \forall t \in \{1,2,\dots,T\}$
4. Gets utility (payoff) $\beta^{t-1}\sqrt{C}$ of consuming $C$ apples in period $t$ 

She is solving the following problem:

$$
\begin{aligned}
V_T^{\star}(M) &= \max_{C_1,C_2,\dots,C_T} \{ \sqrt{C_1} +\beta \sqrt{C_2}+\beta^2\sqrt{C_3}+\cdots+\beta^T\sqrt{C_T} \} \\
    & \text{s.t.} \\
    M &= C_1 + C_2+\cdots+C_T \\
    C_t &\in \mathbb{N}, \, \forall t \in \{1,2,\dots,T\}
\end{aligned}
$$

where $V_T^{\star}(M)$ is called the **value function**, $M$ is the **state variable**, $\{C_t\}_{t=1}^{T}$ are the **control variables**.

## Code

We set up our model using `SimpleNamespace()`, which is simply a dictionary, where the value to a key is accessed with `model.key` instead of `model['key']`.

In [2]:
model = SimpleNamespace() # model
model.par = SimpleNamespace() # parameters
model.sol = SimpleNamespace() # solution
model.sim = SimpleNamespace() # simulation

Let the parameters of interest be:

In [3]:
model.par.T = 4 # number of time periods
model.par.M = 6 # number of apples
model.par.beta = 0.90 # discount factor

## Brute-force solution

A brute-force solution is just to try out all posible choices:

In [4]:
def solve_by_brute_force(model,utility):
    """ solve the model using a brute force approach """
        
    par = model.par
    sol = model.sol
    
    # a. initilize
    best_value = -np.inf
    
    # b. loop through all possible choices
    for Cs in it.product(par.grid_C,repeat=par.T):
        
        # Cs is [C1,C2,...,CT] where all C1,C2,...,CT are in par.grid_C
        
        # a. feasibility
        if np.sum(Cs) > par.M: continue # consumption larger than ressources
            
        # b. value-of-choice
        discount_factors = model.par.beta**np.arange(0,par.T) 
        value = np.sum(discount_factors*utility(np.array(Cs)))
        
        # c. best?
        if value > best_value:        
            best_value = value
            best_Cs = Cs
            
    # c. save
    sol.Cs = np.array(best_Cs)
    sol.V = best_value

**Grid of possible choices in each period:**

In [5]:
model.par.grid_C = np.arange(model.par.M+1) # 0,1,...,M
print(model.par.grid_C)

[0 1 2 3 4 5 6]


**Results:**

In [6]:
solve_by_brute_force(model,utility=np.sqrt)
print(f'best value = {model.sol.V:.3f} with Cs = {model.sol.Cs}')

best value = 4.226 with Cs = [2 2 1 1]


## Backwards inducation

>**Bellman's principle of Optimality:** *An optimal policy has the property that whatever the initial state and initial decision are, the remaining decisions must constitute an optimal policy with regard to the state resulting from the first decision.* 

Mathematically we obviously have the recursion:

$$
\begin{aligned}
V_{T}^{\star}(M) &= \sqrt{M} \\
V_{T-1}^{\star}(M) &= \max_{C_{T-1}}\left\{ \sqrt{C_{T-1}}+\beta V_{T}^{\star}(M-C_{T-1})\right\} \\
V_{T-2}^{\star}(M) &= \max_{C_{T-2}}\left\{ \sqrt{C_{T-2}}+\beta V_{T-1}^{\star}(M-C_{T-2})\right\} \\
V_{T-3}^{\star}(M) &= \max_{C_{T-3}}\left\{ \sqrt{C_{T-3}}+\beta V_{T-2}^{\star}(M-C_{T-3})\right\} \\
& \dots \\
V_{t}^{\star}(M) &= \max_{C_{t}}\left\{ \sqrt{C_{t}}+\beta V_{t+1}^{\star}(M-C_{t})\right\}  \\
\end{aligned}
$$

where $\beta V_{t+1}^{\star}(M-C_{t})$ is called the **continuation value**. 

The final equation is called the **Bellman equation**. The consumption function in each period, $C_t^{\star}(M_t)$ is called the **policy function**.

Using **backwards induction** we can now solve for the value and policy functions $\{V_t^{\star}(M_t)\}_{t=1}^T$ and $\{C_t^{\star}(M_t)\}_{t=1}^T$ by for each $t \in \{T,T-1,\dots,1\}$ solving the problem

$$
\begin{aligned}
V_{t}^{\star}(M_t) &= \max_{C_{t}} \sqrt{C_{t}}+\beta V_{t+1}^{\star}(M_{t+1})  \\
    & \text{s.t.} \\
    M_{t+1} &= M_t-C_t \\
    C_t &\in \{0,1\dots,M_t\}
\end{aligned}
$$

where the state variable $M_t$ is restricted to lie in the grid $\{0,1,2,\dots,M\}$, and $M_{t+1} = \Gamma(M_t,C_t) = M_t-C_t$ is called the **transition rule** (or transition function).

In [7]:
def solve_time_step(t,model,utility):
    """ solve problem in time step t """
    
    par = model.par
    sol = model.sol
    
    # loop over grid
    for i_M,M in enumerate(model.par.grid_M):
        
        best_value = -np.inf
        best_C = np.nan
        for C in par.grid_C:
            
            # a. feasibility
            if C > M: continue # consumption larger than ressources
            
            # b. value-of-choice
            M_plus = M-C
            value = utility(C) + par.beta*sol.V[t+1,np.int_(M_plus)]
            
            # c. best?
            if value > best_value:
                best_value = value
                best_C = C
        
        # save best
        sol.V[t,i_M] = best_value
        sol.C[t,i_M] = best_C
    
def solve_by_backwards_induction(model,utility):
    """ solve the model by backwards induction """
    
    par = model.par
    sol = model.sol    
    
    # a. allocate
    sol.C = np.zeros((par.T,par.grid_M.size))
    sol.V = np.zeros((par.T,par.grid_M.size))
    
    # b. time loop
    for t in reversed(range(par.T)):
        
        if t == par.T-1: # last period
            sol.C[t,:] = par.grid_M
            sol.V[t,:] = utility(par.grid_M)
        else: # before last period
            solve_time_step(t,model,utility)

**Solve problem:**

In [8]:
model.par.grid_M = np.arange(model.par.M+1)
solve_by_backwards_induction(model,utility=np.sqrt)

**Simulate model for specific M:**

In [9]:
def simulate(model):
    """ simulate the model """
    
    par = model.par
    sol = model.sol
    sim = model.sim
    
    # a. allocate
    sim.M = np.zeros(par.T,np.int_)
    sim.C = np.zeros(par.T,np.int_)
    
    # b. simulate
    sim.M[0] = par.M
    for t in range(par.T):
        sim.C[t] = sol.C[t,sim.M[t]]
        if t < par.T-1:
            sim.M[t+1] = sim.M[t]-sim.C[t]

In [10]:
simulate(model)
print(f'best value {model.sol.V[0,model.par.M]:.3f}, Cs = {model.sim.C}')    

best value 4.226, Cs = [2 2 1 1]


## Continous choices

**Question:** What if $C_t \in \mathbb{R}_+$ is allowed? (i.e. not just natural numbers)

**Answer with brute-force:** Same brute-force solution method can be used, but it now depends on the density of the grid for $C_t$. And becomes very costly for dense grids.

In [11]:
for NC in [10,20,30]:
    
    t0 = time.time()
    
    model.par.grid_C = np.linspace(0,model.par.M,NC)
    solve_by_brute_force(model,utility=np.sqrt)
    
    print(f'best value = {model.sol.V:.3f} with Cs = {model.sol.Cs}')
    print(f'found in {elapsed(t0)}\n')

best value = 4.231 with Cs = [2.         1.33333333 1.33333333 1.33333333]
found in 0.1 secs

best value = 4.238 with Cs = [2.21052632 1.57894737 1.26315789 0.94736842]
found in 1.0 secs

best value = 4.240 with Cs = [2.06896552 1.65517241 1.24137931 1.03448276]
found in 4.8 secs



**Answer with backwards interation:**

Here the best solution involes a combination of:

1. **Linear interpolation:** See [here](https://numeconcopenhagen.netlify.com/lectures/Numerical_optimization#Interpolation).
2. **Numerical optimization:** See [here](https://numeconcopenhagen.netlify.com/lectures/Optimize_print_and_plot#Algorithm-3:-Call-a-solver).

The extended code is:

In [12]:
def solve_time_step_cont(t,model,utility):
    """ solve problem in time step t """
    
    par = model.par
    sol = model.sol
    
    # loop over grid
    for i_M,M in enumerate(model.par.grid_M):
        
        # i. objective to minimize
        obj = lambda C: -(utility(C) + par.beta*np.interp(M-C,par.grid_M,sol.V[t+1]))
        
        # ii. minimizer
        result = optimize.minimize_scalar(obj,method='bounded',bounds=(0,M))

        # iii. save best
        sol.V[t,i_M] = -result.fun
        sol.C[t,i_M] = -result.x
    
def solve_by_backwards_induction_cont(model,utility):
    """ solve the model by backwards induction """
    
    par = model.par
    sol = model.sol    
    
    # a. allocate
    sol.C = np.zeros((par.T,par.grid_M.size))
    sol.V = np.zeros((par.T,par.grid_M.size))
    
    # b. time loop
    for t in reversed(range(par.T)):
        
        if t == par.T-1: # last period
            sol.C[t,:] = par.grid_M
            sol.V[t,:] = utility(par.grid_M)
        else: # else
            solve_time_step_cont(t,model,utility)

In [13]:
for NM in [10,20,30]:
    
    t0 = time.time()
    
    model.par.grid_M = np.linspace(0,model.par.M,NM)
    solve_by_backwards_induction_cont(model,utility=np.sqrt)
    
    print(f'best value = {np.interp(model.par.M,model.par.grid_M,model.sol.V[0,:]):.3f}')
    print(f'found in {elapsed(t0)}\n')

best value = 4.232
found in 0.0 secs

best value = 4.239
found in 0.0 secs

best value = 4.240
found in 0.0 secs



## Uncertainty

**Question:** What if there consumer a $\pi$-percent chance that an apple rot each period? I.e. the transition function then becomes: 

$$
M_{t+1} = \Gamma(M_t,C_t) \sim
\begin{cases}
\max\{M_{t}-C_{t}-1,0\} & \text{with prob. }\pi\\
M_{t}-C_{t} & \text{with prob. }1-\pi
\end{cases}
$$

and this is **stochastic**.

**No annswer with brute-force exists:** Information is revealed over time, so a once-and-for-all solution does not exist.

**Answer with backwards inducation:** Straigtforward extension with the Bellman equation

$$
V_{t}^{\star}(M_t) = \max_{C_{t}} \sqrt{C_{t}} + \mathbb{E}_t[\beta V_{t+1}^{\star}(M_{t+1})]
$$

where we the **expected continous value** now needs to be computed using some form of **numerical integration**.

In [14]:
def solve_time_step_cont_stoch(t,model,utility):
    """ solve problem in time step t """
    
    par = model.par
    sol = model.sol
    
    # loop over grid
    for i_M,M in enumerate(model.par.grid_M):
        
        # i. objective to minimize
        def obj(C):
            
            obj = utility(C)
            obj += par.pi*par.beta*np.interp(np.fmax(M-C-1,0),par.grid_M,sol.V[t+1])
            obj += (1-par.pi)*par.beta*np.interp(np.fmax(M-C,0),par.grid_M,sol.V[t+1])
            
            return -obj
        
        # ii. minimizer
        result = optimize.minimize_scalar(obj,method='bounded',bounds=(0,M))

        # iii. save best
        sol.V[t,i_M] = -result.fun
        sol.C[t,i_M] = result.x
    
def solve_by_backwards_induction_cont_stoch(model,utility):
    """ solve the model by backwards induction """
    
    par = model.par
    sol = model.sol    
    
    # a. allocate
    sol.C = np.zeros((par.T,par.grid_M.size))
    sol.V = np.zeros((par.T,par.grid_M.size))
    
    # b. time loop
    for t in reversed(range(par.T)):
        
        if t == par.T-1: # last period
            sol.C[t,:] = par.grid_M
            sol.V[t,:] = utility(par.grid_M)
        else: # else
            solve_time_step_cont_stoch(t,model,utility)

**Results:**

In [15]:
model.par.pi = 0.50 # parameter chioce
for NM in [10,20,30]:
    
    t0 = time.time()
    
    model.par.grid_M = np.linspace(0,model.par.M,NM)
    solve_by_backwards_induction_cont_stoch(model,utility=np.sqrt)
    
    print(f'best value = {np.interp(model.par.M,model.par.grid_M,model.sol.V[0,:]):.3f}')
    print(f'found in {elapsed(t0)}\n')

best value = 3.596
found in 0.0 secs

best value = 3.628
found in 0.0 secs

best value = 3.631
found in 0.0 secs



**Simulate:**

In [16]:
def simulate_stoch(model):
    """ simulate the model """
    
    par = model.par
    sol = model.sol
    sim = model.sim
    
    # a. allocate
    sim.M = np.zeros(par.T)
    sim.C = np.zeros(par.T)
    
    # b. simulate
    sim.M[0] = par.M
    for t in range(par.T):
        sim.C[t] = np.interp(sim.M[t],par.grid_M,sol.C[t])
        if t < par.T-1:
            if np.random.uniform(0,1) < par.pi:
                sim.M[t+1] = np.fmax(sim.M[t]-sim.C[t]-1,0)
            else:
                sim.M[t+1] = sim.M[t]-sim.C[t]

**Run simulation:**

In [17]:
np.random.seed(1917)
for _ in range(10):
    simulate_stoch(model)
    print('[',end='')
    for t in range(model.par.T):
        print(f'{model.sim.C[t]:.3f} ',end='')
    print(']')    

[1.483 1.241 1.069 0.000 ]
[1.483 1.241 0.862 1.414 ]
[1.483 1.241 1.069 0.000 ]
[1.483 1.269 1.434 0.814 ]
[1.483 1.241 1.069 0.207 ]
[1.483 1.269 0.838 0.410 ]
[1.483 1.241 1.069 0.000 ]
[1.483 1.269 1.434 0.814 ]
[1.483 1.241 0.862 1.414 ]
[1.483 1.269 1.434 1.814 ]


# Further issues

Solving a dynamic optimization problem with backwards induction is also called **value function iteration**.

1. The same method works if there are **multiple states and choices**
2. The states and choices can be **mixed discrete and continuous variables**
3. **Infinite horizon** problems for $T \rightarrow \infty$ is of special interest (the Bellman operator can be a **contraction mapping**)
4. There are many different **interpolation** approaches (splines, regression, etc.) (see [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html))
5. There are many different **numerical integration** approaches (Monte Carlo, equiprobable, Gauss-Hermite, Tauchen etc.) (see [wikipedia](https://en.wikipedia.org/wiki/Numerical_integration))
6. Solution methods using information from **first order conditions** are typically more computationally efficient.
7. Methods for limiting the number of grid points (e.g. using **sparse grids**) is another avenue for speed-up